In [1]:
import itertools
from parlay_system import *

In [2]:
vcu8 =  MoneyLine(event="vcu8", bet_amount=100, odds=-114)
ucf9 =  MoneyLine(event="ucf9", bet_amount=100, odds=-104)

virginiatech4 = MoneyLine(event="virginiatech4", bet_amount=100, odds=-530)
stlouis13 = MoneyLine(event="stlouis13", bet_amount=100, odds=410)

missst5 = MoneyLine(event="missst5", bet_amount=100, odds=-345)
liberty12 = MoneyLine(event="liberty12", bet_amount=100, odds=280)

michiganst2 = MoneyLine(event="michiganst2", bet_amount=100, odds=-2500)
bradley15 = MoneyLine(event="bradley15", bet_amount=100, odds=1300)

louisville7 = MoneyLine(event="louisville7", bet_amount=100, odds=-225)
minnesota10 = MoneyLine(event="minnesota10", bet_amount=100, odds=185)

lsu3 = MoneyLine(event="lsu3", bet_amount=100, odds=-335)
yale14 = MoneyLine(event="yale14", bet_amount=100, odds=275)


# maryland6 = MoneyLine(event="maryland6", bet_amount=100, odds=220)
# belmont11 = MoneyLine(event="belmont11", bet_amount=100, odds=220)

east_region_binaries = [
    [vcu8, ucf9],
    [virginiatech4, stlouis13],
    [missst5,liberty12],
    [michiganst2, bradley15],
    [louisville7, minnesota10],
    [lsu3, yale14],
]

east_region_results = [
    [0, 1],
    [1, 0],
    [0, 1],
    [1, 0],
    [0, 1],
    [1, 0],
]

def create_dicts(in_binaries, in_results):
    ml_lookup = {}
    ml_win_lookup = {}
    id = 0
    for bi in range(len(in_binaries)):
        binary = in_binaries[bi]
        for ml_i in range(len(binary)):
            ml = binary[ml_i]
            ml.set_index(id) 
            is_winner = in_results[bi][ml_i]
            ml_lookup[id] = ml.event
            ml_win_lookup[id] = is_winner
            id += 1
    
    return ml_lookup, ml_win_lookup

ml_lookup, ml_outcome_lookup = create_dicts(east_region_binaries, east_region_results)

In [3]:
ml_lookup

{0: 'vcu8',
 1: 'ucf9',
 2: 'virginiatech4',
 3: 'stlouis13',
 4: 'missst5',
 5: 'liberty12',
 6: 'michiganst2',
 7: 'bradley15',
 8: 'louisville7',
 9: 'minnesota10',
 10: 'lsu3',
 11: 'yale14'}

In [4]:
ml_outcome_lookup

{0: 0, 1: 1, 2: 1, 3: 0, 4: 0, 5: 1, 6: 1, 7: 0, 8: 0, 9: 1, 10: 1, 11: 0}

In [5]:
def findsubsets(s, n):
    return list(itertools.combinations(s, n))

TOURNEY_SIZE = len(east_region_binaries)
SELECT_NUM = 3
s = set()
n = SELECT_NUM

for i in range(TOURNEY_SIZE):
    s.add(i)

all_subsets = findsubsets(s, n)

In [6]:
all_subsets

[(0, 1, 2),
 (0, 1, 3),
 (0, 1, 4),
 (0, 1, 5),
 (0, 2, 3),
 (0, 2, 4),
 (0, 2, 5),
 (0, 3, 4),
 (0, 3, 5),
 (0, 4, 5),
 (1, 2, 3),
 (1, 2, 4),
 (1, 2, 5),
 (1, 3, 4),
 (1, 3, 5),
 (1, 4, 5),
 (2, 3, 4),
 (2, 3, 5),
 (2, 4, 5),
 (3, 4, 5)]

In [7]:
csv = []
df = pd.DataFrame()

for i in range(len(all_subsets)):
    current_subset = all_subsets[i]
    parlay_binaries = []
    for i in range(len(current_subset)):
        index = current_subset[i]
        parlay_binaries.append(east_region_binaries[index])

    ps = ParlaySystem(binaries=parlay_binaries,
                    target_profit=0.50,
                    bounds=(0.01, 30),
                    binary_index_arr=list(current_subset),
                    binary_results_arr=east_region_results,
                    index_to_ml=ml_lookup,
                    index_to_outcome=ml_outcome_lookup
                    )
    csv_res, raw_df = ps.slsqp_solver()
    df = pd.concat([df, raw_df])
    csv.append(csv_res)

     fun: -0.32880862821789325
     jac: array([ 0.51523035, -0.18490566, -0.66858314, -3.6725    ,  0.49906528,
       -0.23252903, -0.73793896, -3.87682692])
 message: 'Positive directional derivative for linesearch'
    nfev: 110
     nit: 11
    njev: 7
  status: 8
 success: False
       x: array([0.01      , 0.11102423, 0.07650895, 0.02596986, 0.0165762 ,
       0.1066319 , 0.07321909, 0.02485316])
slsqp_solver: 
                           event  index_arr     result  event_status   bet  \
0    vcu8_virginiatech4_missst5_  [0, 2, 4]  [0, 1, 0]         False  0.01   
4    ucf9_virginiatech4_missst5_  [1, 2, 4]  [1, 1, 0]         False  0.02   
1  vcu8_virginiatech4_liberty12_  [0, 2, 5]  [0, 1, 1]         False  0.11   
2        vcu8_stlouis13_missst5_  [0, 3, 4]  [0, 0, 0]         False  0.08   
3      vcu8_stlouis13_liberty12_  [0, 3, 5]  [0, 0, 1]         False  0.03   
5  ucf9_virginiatech4_liberty12_  [1, 2, 5]  [1, 1, 1]          True  0.11   
6        ucf9_stlouis13_missst5_

       0.10796325, 0.09628238, 0.04879809])
slsqp_solver: 
                         event  index_arr     result  event_status   bet  \
0    vcu8_missst5_louisville7_  [0, 4, 8]  [0, 0, 0]         False  0.01   
4    ucf9_missst5_louisville7_  [1, 4, 8]  [1, 0, 0]         False  0.02   
1    vcu8_missst5_minnesota10_  [0, 4, 9]  [0, 0, 1]         False  0.10   
5    ucf9_missst5_minnesota10_  [1, 4, 9]  [1, 0, 1]         False  0.11   
2  vcu8_liberty12_louisville7_  [0, 5, 8]  [0, 1, 0]         False  0.10   
3  vcu8_liberty12_minnesota10_  [0, 5, 9]  [0, 1, 1]         False  0.05   
6  ucf9_liberty12_louisville7_  [1, 5, 8]  [1, 1, 0]         False  0.10   
7  ucf9_liberty12_minnesota10_  [1, 5, 9]  [1, 1, 1]          True  0.05   

   multiplier  payout  profit  
0    3.497443  0.0350   -0.50  
4    3.654589  0.0664   -0.47  
1    6.900725  0.7164    0.18  
5    7.210786  0.7785    0.24  
2   10.303704  1.0366    0.50  
3   20.330000  1.0366    0.50  
6   10.766667  1.0366    0.50  


     fun: -0.32428630445641426
     jac: array([ 0.5981679 ,  0.32878897,  0.05943396, -0.73417453, -0.31274154,
       -1.46850543, -2.62416666, -6.029125  ])
 message: 'Positive directional derivative for linesearch'
    nfev: 289
     nit: 23
    njev: 19
  status: 8
 success: False
       x: array([0.01      , 0.08001318, 0.10377603, 0.07591542, 0.10285163,
       0.05212753, 0.0349115 , 0.01769394])
slsqp_solver: 
                                  event  index_arr     result  event_status  \
0    virginiatech4_missst5_louisville7_  [2, 4, 8]  [1, 0, 0]         False   
1    virginiatech4_missst5_minnesota10_  [2, 4, 9]  [1, 0, 1]         False   
2  virginiatech4_liberty12_louisville7_  [2, 5, 8]  [1, 1, 0]         False   
3  virginiatech4_liberty12_minnesota10_  [2, 5, 9]  [1, 1, 1]          True   
4        stlouis13_missst5_louisville7_  [3, 4, 8]  [0, 0, 0]         False   
5        stlouis13_missst5_minnesota10_  [3, 4, 9]  [0, 0, 1]         False   
6      stlouis13_liberty

       0.03538363, 0.05178992, 0.01793322])
slsqp_solver: 
                               event   index_arr     result  event_status  \
0    virginiatech4_louisville7_lsu3_  [2, 8, 10]  [1, 0, 1]         False   
2    virginiatech4_minnesota10_lsu3_  [2, 9, 10]  [1, 1, 1]          True   
1  virginiatech4_louisville7_yale14_  [2, 8, 11]  [1, 0, 0]         False   
3  virginiatech4_minnesota10_yale14_  [2, 9, 11]  [1, 1, 0]         False   
4        stlouis13_louisville7_lsu3_  [3, 8, 10]  [0, 0, 1]         False   
5      stlouis13_louisville7_yale14_  [3, 8, 11]  [0, 0, 0]         False   
6        stlouis13_minnesota10_lsu3_  [3, 9, 10]  [0, 1, 1]         False   
7      stlouis13_minnesota10_yale14_  [3, 9, 11]  [0, 1, 0]         False   

    bet  multiplier  payout  profit  
0  0.01    2.229513  0.0223   -0.46  
2  0.08    4.399000  0.3528   -0.12  
1  0.10    6.438679  0.6635    0.19  
3  0.08   12.704009  0.9775    0.50  
4  0.10    9.565672  0.9775    0.50  
5  0.04   27.625000

In [8]:
df.head()

,event,index_arr,result,event_status,bet,multiplier,payout,profit
0,vcu8_virginiatech4_missst5_,"[0, 2, 4]","[0, 1, 0]",False,0.01,2.878157,0.0288,-0.42
4,ucf9_virginiatech4_missst5_,"[1, 2, 4]","[1, 1, 0]",False,0.02,3.007478,0.0499,-0.39
1,vcu8_virginiatech4_liberty12_,"[0, 2, 5]","[0, 1, 1]",False,0.11,8.479245,0.9414,0.50
2,vcu8_stlouis13_missst5_,"[0, 3, 4]","[0, 0, 0]",False,0.08,12.348665,0.9448,0.50
3,vcu8_stlouis13_liberty12_,"[0, 3, 5]","[0, 0, 1]",False,0.03,36.380000,0.9448,0.50


In [9]:
ml_lookup[11]

'yale14'

In [28]:
negative_df = df.loc[df.profit < 0]
negative_df = negative_df.sort_values(by=['profit'])
negative_df.describe()

,bet,multiplier,payout,profit
count,36.000000,36.000000,36.000000,36.000000
mean,0.021944,2.895449,0.078336,-0.306389
std,0.023400,0.815691,0.108115,0.147477
min,0.010000,1.594553,0.015900,-0.540000
25%,0.010000,2.297855,0.022975,-0.452500
50%,0.010000,2.887811,0.033600,-0.270000
75%,0.020000,3.503309,0.056475,-0.207500
max,0.090000,4.773426,0.425000,-0.050000


In [29]:
ml_lookup

{0: 'vcu8',
 1: 'ucf9',
 2: 'virginiatech4',
 3: 'stlouis13',
 4: 'missst5',
 5: 'liberty12',
 6: 'michiganst2',
 7: 'bradley15',
 8: 'louisville7',
 9: 'minnesota10',
 10: 'lsu3',
 11: 'yale14'}

In [30]:
negative_df

,event,index_arr,result,event_status,bet,multiplier,payout,profit
0,missst5_louisville7_lsu3_,"[4, 8, 10]","[0, 0, 1]",False,0.01,2.419280,0.0242,-0.54
0,vcu8_missst5_lsu3_,"[0, 4, 10]","[0, 0, 1]",False,0.01,3.144085,0.0314,-0.52
0,vcu8_louisville7_lsu3_,"[0, 8, 10]","[0, 0, 1]",False,0.01,3.520904,0.0352,-0.51
4,ucf9_missst5_lsu3_,"[1, 4, 10]","[1, 0, 1]",False,0.02,3.285354,0.0585,-0.50
0,vcu8_missst5_louisville7_,"[0, 4, 8]","[0, 0, 0]",False,0.01,3.497443,0.0350,-0.50
4,ucf9_louisville7_lsu3_,"[1, 8, 10]","[1, 0, 1]",False,0.02,3.679104,0.0673,-0.48
4,ucf9_missst5_louisville7_,"[1, 4, 8]","[1, 0, 0]",False,0.02,3.654589,0.0664,-0.47
0,virginiatech4_missst5_louisville7_,"[2, 4, 8]","[1, 0, 0]",False,0.01,2.214657,0.0221,-0.46
0,virginiatech4_louisville7_lsu3_,"[2, 8, 10]","[1, 0, 1]",False,0.01,2.229513,0.0223,-0.46
0,virginiatech4_missst5_lsu3_,"[2, 4, 10]","[1, 0, 1]",False,0.01,1.990903,0.0199,-0.45


In [44]:
def color_negative_red(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    if type(val) != bool:
        return 'color: black'
    
    color = 'red' if val == True else 'black'
    return 'color: %s' % color

hl = negative_df.reset_index(drop=True).style.applymap(color_negative_red)

AttributeError: 'Styler' object has no attribute 'reset_index'

In [41]:
hl

,event,index_arr,result,event_status,bet,multiplier,payout,profit
0,missst5_louisville7_lsu3_,"['4', '8', '10']","[0, 0, 1]",False,0.01,2.41928,0.0242,-0.54
1,vcu8_missst5_lsu3_,"['0', '4', '10']","[0, 0, 1]",False,0.01,3.14409,0.0314,-0.52
2,vcu8_louisville7_lsu3_,"['0', '8', '10']","[0, 0, 1]",False,0.01,3.5209,0.0352,-0.51
3,ucf9_missst5_lsu3_,"['1', '4', '10']","[1, 0, 1]",False,0.02,3.28535,0.0585,-0.5
4,vcu8_missst5_louisville7_,"['0', '4', '8']","[0, 0, 0]",False,0.01,3.49744,0.035,-0.5
5,ucf9_louisville7_lsu3_,"['1', '8', '10']","[1, 0, 1]",False,0.02,3.6791,0.0673,-0.48
6,ucf9_missst5_louisville7_,"['1', '4', '8']","[1, 0, 0]",False,0.02,3.65459,0.0664,-0.47
7,virginiatech4_missst5_louisville7_,"['2', '4', '8']","[1, 0, 0]",False,0.01,2.21466,0.0221,-0.46
8,virginiatech4_louisville7_lsu3_,"['2', '8', '10']","[1, 0, 1]",False,0.01,2.22951,0.0223,-0.46
9,virginiatech4_missst5_lsu3_,"['2', '4', '10']","[1, 0, 1]",False,0.01,1.9909,0.0199,-0.45


In [13]:
positive_df = df.loc[df.profit >= 0]
positive_df = positive_df.sort_values(by=['profit'])
positive_df.describe()

,bet,multiplier,payout,profit
count,124.000000,124.000000,124.000000,124.000000
mean,0.054839,37.567874,0.901947,0.524919
std,0.036168,49.241028,0.342813,0.348265
min,0.010000,4.635849,0.352500,0.070000
25%,0.020000,8.831087,0.757300,0.500000
50%,0.050000,19.976250,0.890200,0.500000
75%,0.090000,38.427628,0.979500,0.500000
max,0.120000,271.320000,2.713200,2.430000


In [14]:
ml_lookup

{0: 'vcu8',
 1: 'ucf9',
 2: 'virginiatech4',
 3: 'stlouis13',
 4: 'missst5',
 5: 'liberty12',
 6: 'michiganst2',
 7: 'bradley15',
 8: 'louisville7',
 9: 'minnesota10',
 10: 'lsu3',
 11: 'yale14'}

In [15]:
positive_df

,event,index_arr,result,event_status,bet,multiplier,payout,profit
1,virginiatech4_michiganst2_yale14_,"[2, 6, 11]","[1, 1, 0]",False,0.08,4.635849,0.3525,0.07
2,virginiatech4_liberty12_michiganst2_,"[2, 5, 6]","[1, 1, 1]",True,0.08,4.697660,0.3600,0.08
1,virginiatech4_missst5_yale14_,"[2, 4, 11]","[1, 0, 0]",False,0.10,5.749590,0.5693,0.10
2,virginiatech4_liberty12_lsu3_,"[2, 5, 10]","[1, 1, 1]",True,0.10,5.865334,0.5863,0.11
1,missst5_michiganst2_yale14_,"[4, 6, 11]","[0, 1, 0]",False,0.08,5.030435,0.3913,0.11
...,...,...,...,...,...,...,...,...
7,liberty12_bradley15_minnesota10_,"[5, 7, 9]","[1, 0, 1]",False,0.01,151.620000,1.5162,1.23
7,liberty12_bradley15_yale14_,"[5, 7, 11]","[1, 0, 0]",False,0.01,199.500000,1.9950,1.71
7,stlouis13_bradley15_minnesota10_,"[3, 7, 9]","[0, 0, 1]",False,0.01,203.490000,2.0349,1.75
7,stlouis13_bradley15_yale14_,"[3, 7, 11]","[0, 0, 0]",False,0.01,267.750000,2.6775,2.40


In [42]:
positive_df.groupby('event_status').count()

,event,index_arr,result,bet,multiplier,payout,profit
event_status,,,,,,,
False,111,111,111,111,111,111,111
True,13,13,13,13,13,13,13


In [43]:
negative_df.groupby('event_status').count()

AttributeError: 'Styler' object has no attribute 'groupby'

In [16]:
csv

[[['1', '3', '5'], 0.45, 0.27375, 0.41901031354507595, 0.5, -0.42],
 [['1', '3', '7'], 0.22, 0.48375, 0.5089186715829105, 1.17, -0.21],
 [['1', '3', '9'], 0.39, 0.20750000000000002, 0.36966200778549047, 0.5, -0.36],
 [['1', '3', '11'], 0.46, 0.27, 0.4197958687607245, 0.5, -0.42],
 [['1', '5', '7'], 0.25, 0.35625, 0.38674234686305414, 0.78, -0.23],
 [['1', '5', '9'], 0.54, 0.18125, 0.4306286932248842, 0.5, -0.5],
 [['1', '5', '11'], 0.56, 0.2475, 0.4675697045055727, 0.5, -0.52],
 [['1', '7', '9'], 0.21, 0.26125, 0.31320634458087576, 0.57, -0.18],
 [['1', '7', '11'], 0.25, 0.35125, 0.38353385024755543, 0.77, -0.23],
 [['1', '9', '11'], 0.55, 0.18, 0.43526674908283863, 0.5, -0.51],
 [['3', '5', '7'], 0.29, 0.5875, 0.8004418422703444, 2.43, -0.27],
 [['3', '5', '9'], 0.47, 0.26375, 0.37071118908698414, 0.5, -0.46],
 [['3', '5', '11'], 0.47, 0.2825, 0.3456153931757091, 0.5, -0.45],
 [['3', '7', '9'], 0.28, 0.50875, 0.6020070597592689, 1.75, -0.26],
 [['3', '7', '11'], 0.29, 0.58375, 0.79161

In [17]:
ps.all_parlays[0].money_line_arr[0].event

'michiganst2'

In [18]:
ps.index_to_outcome

{0: 0, 1: 1, 2: 1, 3: 0, 4: 0, 5: 1, 6: 1, 7: 0, 8: 0, 9: 1, 10: 1, 11: 0}